In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import unicode_literals

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import datetime

d1 = datetime.date(2016, 5, 30)
d2 = datetime.date(2016, 7, 8)
delta = d2 - d1
dates = [(d1 + datetime.timedelta(days=i)).strftime('%Y-%m-%d') for i in range(delta.days + 1)]
dates = filter(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').isoweekday() in range(1, 6), dates)

time_from = '09:00:00'
time_to = '18:00:00'

print dates

In [ ]:
raw_data = pd.read_csv('data/sensor_output.csv', names=['timestamp', 'sound', 'motion'], index_col=0, parse_dates = True)
raw_data.index = raw_data.index+pd.offsets.Hour(2)
raw_data = raw_data[(raw_data['sound'] < 1000) & (raw_data['motion'] < 100)]
raw_data.tail()

In [ ]:
calendar_data = pd.read_csv('data/yellow_merged_events.csv', index_col=0, parse_dates = True)
calendar_data = calendar_data[calendar_data.status == 'confirmed']
calendar_data.tail()

In [ ]:
aggregated_data = raw_data.groupby(pd.TimeGrouper('5Min')).agg({'sound' : 'std', 'motion' : 'mean'})
aggregated_data['sound'] = aggregated_data['sound'] - aggregated_data['sound'].mean()
aggregated_data.sound[aggregated_data.sound < 0] = 0

aggregated_data.to_csv('data/sensor_aggregated_data.csv', sep=b',', encoding='utf-8')
aggregated_data.tail()

In [ ]:
font = {'size': 16}
merged_dataframes = pd.DataFrame()

for selected_date in dates:
    datetime_from = selected_date + ' ' + time_from 
    datetime_to = selected_date + ' ' + time_to
    single_day_data = aggregated_data.loc[(aggregated_data.index >= datetime_from) & (aggregated_data.index <= datetime_to),:].copy()
    single_day_data.loc[:,('sound')] = single_day_data.loc[:,('sound')] / single_day_data['sound'].max()
    single_day_data = single_day_data.fillna(0.7)
    single_day_data.sound[single_day_data.sound < 0.3] = 0
    single_day_data.motion[single_day_data.motion < 0.1] = 0
    merged_dataframes = merged_dataframes.append(single_day_data)
    
    plot = single_day_data.plot(figsize=(16,8))
    plt.ylabel('correlated sensor data')
    plt.title('correlated data for ' + selected_date)
    
    single_day_calendar_data = calendar_data[(calendar_data.index >= datetime_from) & (calendar_data.index <= datetime_to)]
    for index, row in single_day_calendar_data.iterrows():
        plot.axvspan(index, row['end_date_time'], facecolor='y', alpha=0.2)
        text = unicode(str(row['summary']), 'utf-8') + ' - ' + row['status']
        plot.text(index, 0.9, text, rotation=90, fontdict=font)

In [ ]:
merged_dataframes.tail()

In [ ]:
busy_vs_booked_dataframe = merged_dataframes.copy()
busy_vs_booked_dataframe['busy'] = ((busy_vs_booked_dataframe.sound > 0) | (busy_vs_booked_dataframe.motion > 0)).astype(int)
busy_vs_booked_dataframe['booked'] = 0
busy_vs_booked_dataframe['start_date_time'] = busy_vs_booked_dataframe.index

for index, row in calendar_data.iterrows():
    busy_vs_booked_dataframe.loc[(busy_vs_booked_dataframe.index >= index) & (busy_vs_booked_dataframe.index < row.end_date_time),'booked'] = 1
    busy_vs_booked_dataframe.loc[(busy_vs_booked_dataframe.index >= index) & (busy_vs_booked_dataframe.index < row.end_date_time),'summary'] = row.summary
    busy_vs_booked_dataframe.loc[(busy_vs_booked_dataframe.index >= index) & (busy_vs_booked_dataframe.index < row.end_date_time),'organizer'] = row.organizer
    busy_vs_booked_dataframe.loc[(busy_vs_booked_dataframe.index >= index) & (busy_vs_booked_dataframe.index < row.end_date_time),'recurring_event_id'] = row.recurring_event_id
    busy_vs_booked_dataframe.loc[(busy_vs_booked_dataframe.index >= index) & (busy_vs_booked_dataframe.index < row.end_date_time),'id'] = row.id

busy_vs_booked_dataframe.to_csv('data/yellow_busy_vs_booked.csv', sep=b',', encoding='utf-8')
busy_vs_booked_dataframe.tail()